In [1]:
import requests
import argparse
from bs4 import BeautifulSoup
from tqdm import tqdm
import io
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

from multiprocessing import Pool
from PIL import Image

from repalette.constants import BASE_DATA_DIR, RAW_DATA_DIR, DATA_DIR, DATABASE_PATH
from repalette.utils.models import RawImage
from repalette.utils.models import Base

In [2]:
DESIGN_SEEDS_PAGES_ROOT = r"https://www.design-seeds.com/blog/page/"

In [3]:
from repalette.utils.data_preparation import get_image_urls_and_palettes, get_image_name

In [4]:
image_urls = []
palettes = []

i = 0
bar = tqdm(desc="Pages downloaded")
response = requests.get(DESIGN_SEEDS_PAGES_ROOT + str(i))
bar.update(n=1)
while response.status_code != 404:
    bs = BeautifulSoup(response.content, "html.parser")
    posts = bs.find_all(class_="entry-content")

    for post in posts:
        image_url = post.find_all(class_="attachment-full")[0]["src"]
        palette = [header.text for header in post.find_all("h5")[1:]]

        image_urls.append(image_url)
        palettes.append(palette)
        
    bar.update(n=1)

    i += 1
    response = requests.get(DESIGN_SEEDS_PAGES_ROOT + str(i))
    
    if i == 5:
        break

Pages downloaded: 6it [00:06,  1.23s/it]

In [4]:
if not os.path.exists(BASE_DATA_DIR):
    os.mkdir(BASE_DATA_DIR)
if not os.path.exists(RAW_DATA_DIR):
    os.mkdir(RAW_DATA_DIR)
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

engine = create_engine(f"sqlite:///{DATABASE_PATH}")
# create a configured "Session" class
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)

In [6]:
def download_image_to_database(image):
    image_url, palette = image
    image_name = get_image_name(image_url)
    image_save_path = os.path.join(DATA_DIR, image_name)

    # create a database Session
    session = Session()

    raw_image = RawImage(
        path=image_save_path,
        palette=palette,
        url=image_url,
        name=image_name,
    )
    try:
        session.add(raw_image)
        # if add successful (new image) - download image
        image_data = requests.get(image_url).content
        image = Image.open(io.BytesIO(image_data))
        raw_image.height = image.height
        raw_image.width = image.width
        session.commit()
        # save image on disk
        image.save(image_save_path, "PNG")

    except IntegrityError:  # image already in the database
        pass


Pages downloaded: 6it [00:13,  2.21s/it]A

100%|██████████| 30/30 [00:08<00:00,  3.42it/s]


In [5]:
session = Session()
raw_images = session.query(RawImage).all()

In [6]:
raw_images